In [ ]:
import pandas as pd
import time
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager

HEADER = 'headless'

options = webdriver.ChromeOptions()
options.add_argument(HEADER)
options.add_argument('window-size=1920x1080')
options.add_argument("disable-gpu")
options.add_argument("User-Agent:Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/103.0.0.0 Safari/537.36")
options.add_argument("lang=ko_KR")

chromedriver = '/usr/local/chromedriver/chromedriver'
driver = webdriver.Chrome(service=Service(chromedriver), options=options)

# 크롤링할 사이트
# https://www.oliveyoung.co.kr/store/goods/getGoodsDetail.do?goodsNo=A000000117745&dispCatNo=1000001001000010001&trackingCd=Result_11_30
# 테스트용 사이트1 (18건) 되는지 안 되는지 정도만 체크
# https://www.oliveyoung.co.kr/store/goods/getGoodsDetail.do?goodsNo=A000000155413&dispCatNo=100000100080008&trackingCd=Cat100000100080008_Small
# 테스트용 사이트2 (68건) 페이지 잘 넘어가는지
# https://www.oliveyoung.co.kr/store/goods/getGoodsDetail.do?goodsNo=A000000116600&dispCatNo=100000100080008&trackingCd=Cat100000100080008_Small
# 테스트용 사이트3 (118건) 페이지 잘 넘어가고 + 다음 화살표 눌러지는지
# https://www.oliveyoung.co.kr/store/goods/getGoodsDetail.do?goodsNo=A000000152514&dispCatNo=100000100080008&trackingCd=Cat100000100080008_Small
# 테스트용 사이트4 (999건) 페이지 잘 넘어가고 + 다음 화살표 여러 개 눌러지는지
# https://www.oliveyoung.co.kr/store/goods/getGoodsDetail.do?goodsNo=A000000157294&dispCatNo=100000100080008&trackingCd=Cat100000100080008_Small
driver.get('https://www.oliveyoung.co.kr/store/goods/getGoodsDetail.do?goodsNo=A000000157294&dispCatNo=100000100080008&trackingCd=Cat100000100080008_Small', )

# 리뷰 페이지로 이동
WebDriverWait(driver, 60).until(EC.presence_of_element_located((By.CSS_SELECTOR,"#reviewInfo > a"))).click()

names, dates, reviews = [], [], []
CLICKED_NEXT = False
paging_CSS_SELECTOR = 2
while True:
    elems_page = WebDriverWait(driver, 60).until(EC.visibility_of_all_elements_located((By.CSS_SELECTOR, "div.pageing > a")))
    pages = []
    print(type(elems_page[0])) # 테스트용

    for elem in elems_page:
        print('elem : ', elem.get_attribute("data-page-no")) # 테스트용
        pages.append(elem.get_attribute("data-page-no"))
    pages = list(map(int, pages)) # pages에 페이지 번호들이 저장됨

    # 마지막 페이지가 화살표일 경우
    if pages[-1]%10 == 1:
        print("화살표가 있는 경우") # 테스트용
        for page in pages:
            print("page : ", page)
            
            if page == pages[-1]:
                print("화살표 누르기") # 테스트용
                driver.execute_script("arguments[0].click();", (WebDriverWait(driver, 60).until(EC.presence_of_element_located((By.CSS_SELECTOR, "#gdasContentsArea > div > div.pageing > a.next")))))
                # WebDriverWait(driver, 60).until(EC.presence_of_element_located((By.CSS_SELECTOR,"#gdasContentsArea > div > div.pageing > a.next"))).click()
                CLICKED_NEXT = True
                paging_CSS_SELECTOR = 3
                print("넘어갈 때 page 값 : ", page)
                break
            
            # def find_num_of_review:
            elems_review = WebDriverWait(driver, 60).until(EC.visibility_of_all_elements_located((By.CSS_SELECTOR, "div.review_cont")))
            print("리뷰 수 : ", len(elems_review))

            for i in range((len(elems_review))):
                name = WebDriverWait(driver, 60).until(EC.presence_of_element_located((By.CSS_SELECTOR, "#gdasList > li:nth-child(%d) > div.info > div > p > a"%(i+1))))
                date = WebDriverWait(driver, 60).until(EC.presence_of_element_located((By.CSS_SELECTOR, "#gdasList > li:nth-child(%d) > div.review_cont > div.score_area > span.date"%(i+1))))
                review = WebDriverWait(driver, 60).until(EC.presence_of_element_located((By.CSS_SELECTOR, "#gdasList > li:nth-child(%d) > div.review_cont > div.txt_inner"%(i+1))))
                
                time.sleep(5)
                # driver.implicitly_wait(10) # 요소를 찾을 수 없다는 에러가 나는 구간
                names.append(name.text)
                # driver.implicitly_wait(10)                
                dates.append(date.text)
                # driver.implicitly_wait(10)                
                reviews.append(review.text)
            
            if page != pages[-1]:
                if CLICKED_NEXT == True:
                    print("CLICKED_NEXT : True")
                    print("paging_CSS_SELECTOR :", paging_CSS_SELECTOR)
                    driver.execute_script("arguments[0].click();", (WebDriverWait(driver, 60).until(EC.presence_of_element_located((By.CSS_SELECTOR, "#gdasContentsArea > div > div.pageing > a:nth-child(%d)"%(paging_CSS_SELECTOR))))))
                    paging_CSS_SELECTOR += 1
                    
                elif CLICKED_NEXT == False:
                    print("CLICKED_NEXT : False")
                    driver.execute_script("arguments[0].click();", (WebDriverWait(driver, 60).until(EC.presence_of_element_located((By.CSS_SELECTOR, "#gdasContentsArea > div > div.pageing > a:nth-child(%d)"%(page))))))
            
    elif pages[-1]%10 != 1:
        print("화살표가 없는 경우") # 테스트용
        for page in pages:
            print("page : ", page)  # 테스트용
            # def find_num_of_review:
            elems_review = WebDriverWait(driver, 60).until(EC.visibility_of_all_elements_located((By.CSS_SELECTOR, "div.review_cont")))
            print("리뷰 수 : ", len(elems_review))

            for i in range((len(elems_review))):
                name = WebDriverWait(driver, 60).until(EC.presence_of_element_located((By.CSS_SELECTOR, "#gdasList > li:nth-child(%d) > div.info > div > p > a"%(i+1))))
                date = WebDriverWait(driver, 60).until(EC.presence_of_element_located((By.CSS_SELECTOR, "#gdasList > li:nth-child(%d) > div.review_cont > div.score_area > span.date"%(i+1))))
                review = WebDriverWait(driver, 60).until(EC.presence_of_element_located((By.CSS_SELECTOR, "#gdasList > li:nth-child(%d) > div.review_cont > div.txt_inner"%(i+1))))

                # time.sleep(5) # 요소를 찾을 수 없다는 에러가 나는 구간
                names.append(name.text)
                dates.append(date.text)
                reviews.append(review.text)
        if pages[-1] == page:
            break

driver.quit()